In [11]:
import enoki as ek
import mitsuba
mitsuba.set_variant('gpu_autodiff_rgb')

from mitsuba.core import Thread, Float
from mitsuba.core.xml import load_file
from mitsuba.python.util import traverse
from mitsuba.python.autodiff import render, write_bitmap

# Load the Cornell Box
Thread.thread().file_resolver().append('cbox/')

scene = load_file('cbox/cbox_path.xml')
# scene = load_file('cbox/cbox_direct.xml')
# scene = load_file('cbox/cbox_path_tc.xml')
# scene = load_file('cbox/cbox_direct_tc.xml')
# scene = load_file('cbox/cbox_path_nee.xml')

# Find differentiable scene parameters
params = traverse(scene)

wall_res = params['wall_bsdf.reflectance.resolution']
wall_data = Float(params['wall_bsdf.reflectance.data'])
print(wall_res, wall_data)

params.keep(['wall_bsdf.reflectance.data'])

2021-12-03 02:31:34 INFO main [xml.cpp:1221] Loading XML file "cbox\cbox_path.xml" ..

2021-12-03 02:31:34 INFO main [xml.cpp:1222] Using variant "gpu_autodiff_rgb"

2021-12-03 02:31:34 INFO main [xml.cpp:355] "cbox\cbox_path.xml": in-memory version upgrade (v2.0.0 -> v2.2.1) ..

2021-12-03 02:31:34 INFO main [PluginManager] Loading plugin "plugins\path.dll" ..

2021-12-03 02:31:34 INFO main [Scene] Building scene in OptiX ..

[16, 16] [0, 0, 0, 0, 0, .. 758 skipped .., 0.520996, 0.799103, 0.577581, 0.520996, 0.799103]


In [12]:
image_ref = render(scene, spp=256)
crop_size = scene.sensors()[0].film().crop_size()
write_bitmap('out/out_ref.png', image_ref, crop_size)

![out_ref.png](out/out_ref.png)

In [10]:
params['wall_bsdf.reflectance.data'] = ek.full(Float, 1.0, len(wall_data))
params.update()

from mitsuba.python.autodiff import Adam
opt = Adam(params, lr=0.1)

image = render(scene, spp=8)
crop_size = scene.sensors()[0].film().crop_size()
write_bitmap('out/out_start.png', image, crop_size)
write_bitmap('out/wall_start.png', params['wall_bsdf.reflectance.data'],
                 (wall_res[0], wall_res[1]))

for it in range(200):
    # Perform a differentiable rendering of the scene
    image = render(scene, optimizer=opt, unbiased=True, spp=1)

    # Objective: MSE between 'image' and 'image_ref'
    ob_val = ek.hsum(ek.sqr(image - image_ref)) / len(image)

    # Back-propagate errors to input parameters
    ek.backward(ob_val)

    # Optimizer: take a gradient step
    opt.step()

    err_ref = ek.hsum(ek.sqr(wall_data - params['wall_bsdf.reflectance.data']))

    print('%g,%g' % (ob_val[0], err_ref[0]))

image = render(scene, spp=8)
crop_size = scene.sensors()[0].film().crop_size()
write_bitmap('out/out_end.png', image, crop_size)
write_bitmap('out/wall_end.png', params['wall_bsdf.reflectance.data'],
                 (wall_res[0], wall_res[1]))

0.0123962,291.515
0.0101106,234.03
0.0100393,186.971
0.0100392,150.113
0.00813649,122.271
0.00800641,101.879
0.00779011,87.2621
0.00727437,76.8979
0.00744481,69.5705
0.00685016,64.1221
0.00827224,59.938
0.00689399,56.4242
0.00682837,53.2265
0.00735488,50.1129
0.00721989,47.0815
0.00696949,44.0905
0.007148,41.1565
0.00693262,38.3752
0.00681305,35.8346
0.00792719,33.5164
0.0072638,31.4482
0.0064417,29.6263
0.00742215,27.9916
0.00802683,26.5537
0.00751841,25.1897
0.00665254,23.8343
0.00779234,22.3899
0.00760332,20.9238
0.00807677,19.4399
0.00724244,17.9465
0.00825399,16.5705
0.00770868,15.3512
0.00637349,14.294
0.00782198,13.4414
0.00770962,12.7481
0.0071323,12.2217
0.00675264,11.83
0.00894792,11.5151
0.00613237,11.2605
0.00651228,10.9894
0.00864647,10.6622
0.00720148,10.2614
0.00694952,9.7974
0.00737372,9.31338
0.00688413,8.83569
0.00735353,8.3601
0.0075053,7.91738
0.00861677,7.53049
0.00730048,7.23917
0.00658248,7.00684
0.00771541,6.80973
0.00716423,6.6445
0.00739225,6.49142
0.00782617,

![start](out/out_start.png)
![end](out/out_end.png)
![start](out/wall_start.png)
![end](out/wall_end.png)